In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Step 1 - get the data

In [ ]:
url = 'https://en.wikipedia.org/wiki/Berlin'

In [ ]:
city_df = pd.DataFrame(columns = ["city","country","population","latitude","longitude"])#.set_index("city")

In [ ]:
def citygrab2(city):
    url = "https://en.wikipedia.org/wiki/"+city
    response = requests.get(url)
    soup_city = BeautifulSoup(response.content, 'html.parser')
    for header in soup_city.find_all(class_="infobox-header"):
        if header.find(string=re.compile("population", re.IGNORECASE)):
            popu = header.find_next(class_="infobox-data").text

#soup_city.find_all(class_="mergedtoprow")
    for merger in soup_city.find_all(class_="mergedtoprow"):
        if merger.find(string=re.compile("country", re.IGNORECASE)):
            country = merger.find_next(class_="infobox-data").text

    for merger in soup_city.find_all(class_="mergedrow"):
        if merger.find(string=re.compile("country", re.IGNORECASE)):
            country = merger.find_next(class_="infobox-data").text
    #soup_city.find_all(class_="infobox-full-data")

    for coords in soup_city.find_all(class_="infobox-full-data"):
        if coords.find(string=re.compile("Coordinates", re.IGNORECASE)):
            lati = coords.find_next(class_="latitude").text
            longi = coords.find_next(class_="longitude").text

    city_df.loc[len(city_df)]=({"city":str.capitalize(city), "country":country,"population":popu,"latitude":lati,"longitude":longi})
    return city_df

In [ ]:
citygrab2("Berlin")
city = citygrab2("London")

In [ ]:
city

,city,country,population,latitude,longitude
0,Berlin,Germany,"3,755,251",52°31′12″N,13°24′18″E
1,London,England,"8,799,800[1]",51°30′26″N,0°7′39″W


In [ ]:
city['population'] = city['population'].str.replace('[1]','')
city['population'] = city['population'].str.replace(',','')
city["population"] = pd.to_numeric(city["population"])
city

,city,country,population,latitude,longitude
0,Berlin,Germany,3755251,52°31′12″N,13°24′18″E
1,London,England,8799800,51°30′26″N,0°7′39″W


### Step 2 - create a conection string


In [ ]:
import my_stuff

In [ ]:
schema = "gans"
host = "127.0.0.1"
user = "root"
password = my_stuff.my_sql_password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

### Step 3 - Send data

In [ ]:
# city.to_sql(name='city',if_exists='append',con=connection_string,index=False)

2

## AIRPORTS

In [ ]:
cities_df = pd.read_sql('city',con=connection_string)
cities_df

,city_id,city,country,population,latitude,longitude
0,1,Berlin,Germany,3755251,52°31′12″N,13°24′18″E
1,2,London,England,8799800,51°30′26″N,0°7′39″W


In [ ]:
airports_dict = {'ICAO':[],'city_id':[]}

icao_dict = { 'Berlin':'EDDB',
             'London':'EGLL' }

In [ ]:
for i, row in cities_df.iterrows():
    airports_dict['city_id'].append(row['city_id'])
    airports_dict['ICAO'].append(icao_dict[row['city']])

In [ ]:
airports_dict

{'ICAO': ['EDDB', 'EGLL'], 'city_id': [1, 2]}

In [ ]:
airports_df = pd.DataFrame(airports_dict)

In [ ]:
airports_df

,ICAO,city_id
0,EDDB,1
1,EGLL,2


In [ ]:
# airports_df.to_sql('airport',if_exists='append',index=False, con=connection_string)

2